### U.S. CDC Chronic Disease Indicators (CDI) Analysis
June 21, 2018

This analysis is from the CDC U.S. Chronic Disease Indicators (CDI) Analysis https://catalog.data.gov/dataset/u-s-chronic-disease-indicators-cdi
  
Overview of data: https://www.cdc.gov/cdi/overview.html

##### Download the demo files

For this exercise, after downloading the file, remove both the `.`'s and the double underscore from the file name so it looks like:  
`US_Chronic_Disease_Indicators_CDI.csv`.

I also moved "US" to the end of the file name so my working file name is:  
`Chronic_Disease_Indicators_CDI_US.csv`

##### Interact with your file system through the notebook

In [ ]:
import os

In [ ]:
ls

In [ ]:
!conda list

##### Import your Libraries

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##### Read in your CSV file using Pandas

In [2]:
df_cdi = pd.read_csv('Chronic_Disease_Indicators_CDI_US.csv')

In [ ]:
df_cdi = pd.read_csv('Chronic_Disease_Indicators_CDI_US.csv', low_memory=False)

In [ ]:
#data = 'filepath/Datasets'
df_cdi = pd.read_csv('Chronic_Disease_Indicators_CDI_US.csv', encoding='latin1', dtype=object)

%time

## Manipulating Data in Pandas

### Familiarize yourself with the dataset
##### Find the shape of the dataframe

In [3]:
df_cdi.shape

This dataframe is 523,486 rows and 34 columns. That's not huge, but a little big for what working quickly today. 

##### Let's look at the first 5 rows of the dataframe

In [4]:
df_cdi.head()

#### And also the last 5 rows:

In [ ]:
df_cdi.tail()

#### Take a slice of the dataframe to print first two rows of the dataframe

In [ ]:
df_cdi[0:2]

#### Let's use a smaller dataset - Arthritis

Select a subset of rows from the dataframe where the `Topic` column equals 'Arthritis'

In [ ]:
df_arthritis = df_cdi.loc[df_cdi.Topic == 'Arthritis']

#### Inspect the new dataframe

In [ ]:
df_arthritis.shape

#### Write the new DataFrame to a file for later use

In [ ]:
df_arthritis.to_csv('Chronic_Disease_Indicators_Arthritis_US.csv')

#### Load the newly generated file from disk

In [ ]:
df_arth = pd.read_csv('Chronic_Disease_Indicators_Arthritis_US.csv')

#### Make a copy of your working dataset to keep your original data intact

If we import our dataframe as `df` and do not make a copy, any changes we make to `df` will save to (aka overwrite) our original 'Arthritis' subset. If any mistakes are made during this process and we need a fresh start with the original data, without copying, we'll have to start at the very beginning with the full CDC dataset, again subset for 'Arthritis', and continue our analysis from there. Using copy, we're one step ahead with a pristine Arthritis dataset.

If, instead of using `.copy()`, we set  
  
`df = df_arth`

`df` will merely reference `df_arth`. No copy is created here and our original Arthritis subset is vulnerable to changes.

In [ ]:
df = df_arth.copy()

##### Count the number of rows only

In [ ]:
len(df)

##### Count the number of columns only

In [ ]:
len(df.columns)

##### List the columns in your dataset

In [ ]:
df.columns

##### Find the data types of your columns

In [ ]:
df.info()

Now that we have a general idea of the contents and shape of the data, let's move on to preparing the data for analysis.

## Cleaning and Preparing Your Data

### Double check to make sure you're working off a copy of your original dataset so you don't overwrite your original data

### Handling Missing Values

Using the `.count()` function will display the number of values in each column and highlight any columns with missing values. Often these missing values show up in your dataframe as "NaN" - not a number. 

In [ ]:
df.shape

In [ ]:
df.count()

As you can see above, some of the columns are completely empty or have empty cells. Other columns are duplicative. We'll use a subset of eleven columns for today's work:
* **YearStart** - Year the survey was started
* **LocationAbbr** - Abbreviation for state/territory/region name
* **LocationDesc** - Full state/territory/region name
* **Question**
* **DataValueUnit** - All fields use percent (%)
* **DataValueType** - Values: Age-adjusted prevalence, Crude prevalence
* **DataValue** - Integer value
* **DataValueFootnote** - 
* **StratificationCategory1** - Values: Overall, Gender, Race/Ethnicity
* **Stratification1** - Values: Overall; Female; Male; Black, non-Hispanic; Hispanic; Multiracial, non-Hispanic; White, non-Hispanic; Other, non-Hispanic; 
* **GeoLocation** - Latitude, Longitude

Create a dataframe from the original that only contains the columns we want to work with **(copy to Slack and Chat)**. 

In [ ]:
df_sub = df[['YearStart', 'LocationAbbr', 'LocationDesc', 'Question', 'DataValueUnit', 'DataValueType', 'DataValue', 'DatavalueFootnote', 'StratificationCategory1', 'Stratification1', 'GeoLocation']]

In [ ]:
df_sub.head()

We still have missing values. We could replace the NaNs with empty strings for easier text processing, but let's keep them for now. 

In [ ]:
#df_sub = df_sub.fillna('')

In [ ]:
#df_sub.head()

Finally, let's take a look at the data types in our dataset

In [ ]:
df_sub.dtypes

### Exploratory Data Analysis

In [ ]:
df_sub.describe()

In [ ]:
df_sub[['LocationDesc', 'StratificationCategory1']].describe()

#### What questions do we want to answer? What piques our curiosity?
1. What is the overall incidence of arthritis in the U.S. per year?
2. How does this vary by race/ethnicity and gender?
3. Which states had the highest and lowest incidence of arthritis in 2015?


##### 1. What is the overall incidence of arthritis in the U.S. per year?

We can use `pd.unique` to list the unique values in the question column.

In [ ]:
pd.unique(df_sub['Question'])

We can also use `.value_counts()` to count how many times each question appears in our data set. `.value_counts()` is a great tool for exploring categorical data.

In [ ]:
df_sub['Question'].value_counts()

We'll create a subset of our data to answer this question. 

In [ ]:
annual_incidence = df_sub[(df_sub.Question == 'Arthritis among adults aged >= 18 years') & 
                          (df_sub.LocationAbbr == 'US') & (df_sub.StratificationCategory1 == "Overall") & 
                          (df_sub.DataValueType == 'Crude Prevalence')]

In [ ]:
annual_incidence

##### 2. How does this vary by race/ethnicity and gender?

In [ ]:
pd.unique(df_sub['StratificationCategory1'])

In [ ]:
incidence_race = df_sub[(df_sub.Question == 'Arthritis among adults aged >= 18 years') & 
                          (df_sub.LocationAbbr == 'US') & (df_sub.StratificationCategory1 == "Race/Ethnicity")  & 
                          (df_sub.DataValueType == 'Age-adjusted Prevalence')]

In [ ]:
incidence_race

**Why is our output empty?**  
There is no US-wide Race/Ethnicity data for the category Arthritis among adults aged >= 18 years, only for Arthritis among adults aged >= 18 who are obese and only for 2011.

In [ ]:
incidence_race2 = df_sub[(df_sub.LocationAbbr == 'US') & (df_sub.StratificationCategory1 == "Race/Ethnicity") &
                         (df_sub.DataValueType == 'Age-adjusted Prevalence')]

In [ ]:
incidence_race2

##### Let's figure out the answer with data from all the states

We should have 52 readings for each Stratification1 for each year. Let's use `.count()` to check.  

In [ ]:
incidence_race3 = df_sub[(df_sub.Question == 'Arthritis among adults aged >= 18 years') & (df_sub.StratificationCategory1 == "Race/Ethnicity") &
                         (df_sub.DataValueType == 'Age-adjusted Prevalence')].groupby(['YearStart', 'Stratification1'])['DataValue'].count()

In [ ]:
incidence_race3

Let's use the same code from above to get the mean age-adjusted prevalence for each race/ethnicity per year.

In [ ]:
incidence_race4 = df_sub[(df_sub.Question == 'Arthritis among adults aged >= 18 years') & (df_sub.StratificationCategory1 == "Race/Ethnicity") &
                         (df_sub.DataValueType == 'Age-adjusted Prevalence')].groupby(['YearStart', 'Stratification1'])['DataValue'].mean()

**Important Note:** For demo purposes only. In a real-life analysis, you should not take the mean or average of percentages when the total count of individuals across groups is unknown or not the same.  Weighted averages should be used when the totals are known.  

In [ ]:
incidence_race4

##### 3. Which states had the highest and lowest incidence of arthritis in 2015 for adults >= 18 years?

In [ ]:
arthritis_2015 = df_sub[(df_sub.Question == 'Arthritis among adults aged >= 18 years') & (df_sub.YearStart == 2015.)]
len(arthritis_2015)

In [ ]:
mean_incidence_by_state = arthritis_2015.groupby(['LocationAbbr'])['DataValue'].mean()
mean_incidence_by_state

In [ ]:
sorted_values = mean_incidence_by_state.dropna().sort_values()
sorted_values

In [ ]:
min_val = sorted_values.iloc[[0]] # First row in sorted list
min_val

In [ ]:
max_val = sorted_values.iloc[[-1]] # Last row in sorted list
max_val

In [ ]:
min_and_max = sorted_values.iloc[[0, -1]] # First and last row in sorted list
min_and_max

## Making Charts

### Matplotlib

[Matplotlib](https://matplotlib.org/) is a 2D plotting library that serves as the foundation for much of the plotting you will do in Jupyter. It generates decent charts out of the gate and is very useful when conducting an exploratory data analysis.

The [Matplotlib Gallery](https://matplotlib.org/gallery/index.html) provides source code (either as .py file or in a Jupyter Notebook) to create the featured images.

#### matplotlib and pandas in action

Let's subset the `annual_incidence` dataframe that we created above to generate a two-column dataset

In [ ]:
ai = annual_incidence[['YearStart', 'DataValue']].sort_values(by='YearStart')

In [ ]:
ai

In [ ]:
ai.set_index('YearStart')['DataValue'].plot.bar()

##### How does the overall incidence vary by gender?

In [ ]:
gender_incidence = df_sub[(df_sub.Question == 'Arthritis among adults aged >= 18 years') &
                         (df_sub.LocationAbbr == 'US') & 
                         (df_sub.DataValueType == 'Crude Prevalence')].groupby(['YearStart', 'Stratification1'])['DataValue'].mean()

In [ ]:
gender_incidence

In [ ]:
#gif.head()

In [ ]:
gif = gender_incidence.unstack()
gif.plot.bar()

##### Homework: Which states had the highest and lowest incidence of arthritis in 2016?

### Seaborn
[Seaborn](https://seaborn.pydata.org/) is a Python visualization library based on matplotlib that provides nicer graphs. The import convention is:

In [ ]:
import seaborn as sns

In [ ]:
sns.set()

Now that you've imported Seaborn, if you re-run your Matplotlib plots above, they will look nice!

Use Seaborn's [themes](https://seaborn.pydata.org/tutorial/aesthetics.html#aesthetics-tutorial) and [color palettes](https://seaborn.pydata.org/tutorial/color_palettes.html#palette-tutorial) to further decorate your plot.

In [ ]:
sns.set_palette(sns.color_palette("Set2", 10))

[Example Gallery](https://seaborn.pydata.org/examples/index.html#example-gallery)

### Bokeh

[Bokeh](https://bokeh.pydata.org/en/latest/) is a platform for interactive visualization and data applications in modern web browsers. Still under development with occasional breaking changes. The Bokeh Organization includes other projects including [Datashader](https://bokeh.github.io/datashader-docs/) and [HoloViews](http://holoviews.org/).

You can follow Bokeh development on the [Bokeh Blog](https://bokeh.github.io/blog/2017/11/28/release-0-12-11/).

### Altair

[Altair](https://altair-viz.github.io/) is a declarative statistical visualization library for Python developed by Jake VanDerPlas and Brian Granger that is built on top of the powerful [Vega-Lite](https://vega.github.io/vega-lite/) visualization grammar.

### Plotly

[Plotly's](https://plot.ly/) Python graphing library makes really nice interactive, publication-quality graphs online. 